In [2]:
import random
import numpy as np
from tensorflow.keras.utils import to_categorical

In [ ]:
# Imagine there are 10^15 names in this list and we can't load into a list examples anymore!!
# examples = [a.strip() for a in open('names-train.txt') if a.strip()]

In [3]:
# We still need the vocabulary, but can't compute from examples anymore
# Instead, load the examples one by one and add characters as vocabulary items
vocab = set(['<PAD>'])
for a in open('names-train.txt'):
    if a.strip():
        for c in a.strip():
            vocab.add(c)
encoder = dict((c,i) for i,c in enumerate(vocab))
decoder = dict((i,c) for i,c in enumerate(vocab))
inputlen = 10

In [4]:
def example_generator_file(batchsize, filenm, total_examples, enc, inlen):
    ex_file = open(filenm, 'r')
    currexample = ''
    examplenum = 0
    charindex = 0
    Xenc = [enc['<PAD>']]*inlen
    while True:
        X = []
        y = []
        while len(y) < batchsize:  
            while charindex == len(currexample):
                if currexample != '':
                    X.append(Xenc.copy())
                    y.append(enc['<PAD>'])
                currexample = ex_file.readline()
                examplenum += 1
                if currexample == '' or examplenum == total_examples:
                    ex_file.seek(0)
                    currexample = ex_file.readline()
                    examplenum = 0
                currexample = currexample.rstrip('\n')
                charindex = 0
                Xenc = [enc['<PAD>']]*inlen
            nextchar = currexample[charindex]
            X.append(Xenc.copy())
            y.append(enc[nextchar])
            Xenc.pop(0)
            Xenc.append(enc[nextchar])
            charindex += 1
        yield (np.array(X), to_categorical(y, num_classes=len(enc)))

In [ ]:
# below we'll train on full dataset; to separate out development data
# you'll ideally put that data in a second file and call example_generator_file on that
#from sklearn.model_selection import train_test_split
#X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.1, random_state = 42)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
emb_dim = 10
model = Sequential()
model.add(Embedding(input_dim=len(vocab), output_dim=emb_dim, input_length=inputlen))
model.add(Flatten())
model.add(Dense(40))
model.add(Dense(len(vocab), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 10)            540       
_________________________________________________________________
flatten (Flatten)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 40)                4040      
_________________________________________________________________
dense_1 (Dense)              (None, 54)                2214      
Total params: 6,794
Trainable params: 6,794
Non-trainable params: 0
_________________________________________________________________


In [6]:
batch=1024
totalnum=7574
model.fit_generator(example_generator_file(batch, 'names-train.txt', totalnum, encoder, inputlen), steps_per_epoch=totalnum//batch, epochs=200, verbose=1)

Epoch 1/200
7/7 [==============================] - 1s 135ms/step - loss: 3.9736
Epoch 2/200
7/7 [==============================] - 0s 9ms/step - loss: 3.9397
Epoch 3/200
7/7 [==============================] - 0s 7ms/step - loss: 3.9048
Epoch 4/200
7/7 [==============================] - 0s 7ms/step - loss: 3.8609
Epoch 5/200
7/7 [==============================] - 0s 7ms/step - loss: 3.7700
Epoch 6/200
7/7 [==============================] - 0s 7ms/step - loss: 3.7254
Epoch 7/200
7/7 [==============================] - 0s 6ms/step - loss: 3.6434
Epoch 8/200
7/7 [==============================] - 0s 6ms/step - loss: 3.4457
Epoch 9/200
7/7 [==============================] - 0s 6ms/step - loss: 3.2767
Epoch 10/200
7/7 [==============================] - 0s 7ms/step - loss: 3.2191
Epoch 11/200
7/7 [==============================] - 0s 7ms/step - loss: 3.2086
Epoch 12/200
7/7 [==============================] - 0s 7ms/step - loss: 3.0963
Epoch 13/200
7/7 [==============================] - 0s 8ms/

7/7 [==============================] - 0s 7ms/step - loss: 2.4632
Epoch 105/200
7/7 [==============================] - 0s 8ms/step - loss: 2.4940
Epoch 106/200
7/7 [==============================] - 0s 9ms/step - loss: 2.2957
Epoch 107/200
7/7 [==============================] - 0s 7ms/step - loss: 2.3482
Epoch 108/200
7/7 [==============================] - 0s 6ms/step - loss: 2.4733
Epoch 109/200
7/7 [==============================] - 0s 5ms/step - loss: 2.3510
Epoch 110/200
7/7 [==============================] - 0s 7ms/step - loss: 2.2372
Epoch 111/200
7/7 [==============================] - 0s 6ms/step - loss: 2.4749
Epoch 112/200
7/7 [==============================] - 0s 6ms/step - loss: 2.4987
Epoch 113/200
7/7 [==============================] - 0s 5ms/step - loss: 2.4015
Epoch 114/200
7/7 [==============================] - 0s 8ms/step - loss: 2.2313
Epoch 115/200
7/7 [==============================] - 0s 7ms/step - loss: 2.4272
Epoch 116/200
7/7 [==============================] - 0

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
def generate(temperature=1.0):
    answer = ''
    Xout = [encoder['<PAD>']]*inputlen
    while True:
        ydist = model.predict(np.array([Xout]), verbose=0)
        nextchar_index = sample(ydist[0],temperature)
        nextchar = decoder[nextchar_index]
        if nextchar == '<PAD>' or len(answer)>100:
            break
        answer += nextchar
        Xout.append(nextchar_index)
        Xout.pop(0)
    return answer
    

In [17]:
generate(0.5)

'Delile'